<a href="https://colab.research.google.com/github/davidnoone/GEOPHYS_NOTEBOOKS/blob/main/01_Advection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advection, mapping and finite differences

The transport of fluid (atmosphere or ocean) properties by advction is fundemental to a variety of complex behavious. In general it is non-linear, and therefore requires special consideration.

In this example we will evaluate the advection in a 2-dimensional case, to make a prediction for temperature changes. The advection equation is:

$$
  \frac{\partial T}{\partial t} = 
       -u \frac{\partial T}{\partial x}
        - v \frac{\partial T}{\partial y}
$$

We wish to evaluate the temperature change over a 6 hour period. To do so,  the space and time derivarives can be approximated by a "finite difference approach". This notebook develops the skills needed for this and similar dta manipulations.

**Objective:** Create a map of temperature tendency.

**Learning goals:**
 * Utilize notebook framework
 * Read data stored in netcdf format
 * Apply finite difference methods
 * Produce a contour map

**Python modules**
* [numpy](https://numpy.org/doc/stable/user/) for manipulating arrays of gridded data
* [netCDF4](https://unidata.github.io/netcdf4-python/) module for reading NetCDF files
* [Matplotlib](https://matplotlib.org/3.1.1/index.html) library for displaying graphics
* (Bonus: [cartopy](https://scitools.org.uk/cartopy/docs/latest/) for managing fancy map projections)




**Using this notebook**: Remember to save your own copy! Once this note book has loaded, save a copy to your google grive or similar location. Otherwise, you will lose any code work you do when you reload. 

**Notebooks**: Notice that `netCDF4` and `cartopy` is not installed by default on `colab` so must be installed for each session.
Takes a few minutes.


In [ ]:
!pip install netCDF4
!pip install cartopy

In [ ]:
#
# Import the needed modules 
#

import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import cartopy.crs as ccrs

print("All modules needed have been imported. Ready to continue!")

## Obtaining the data

NetCDF is a standard data format which is used widely for atmospehric and oceanic data sets. The data format is structured, and remarkably convinient once the format is appreciated. 

Data sets have dimensions (number of latitude, number of longitudes) and variable (temperature). Some variables descrive the coordinates (i.e., longitudes and latitudes).


In [ ]:
# Download a data file from github, if it doesn't exist
!test ! -f ERA5_pr_2021-01-01.nc && wget 'https://raw.github.com/davidnoone/GEOPHYS_NOTEBOOKS/main/data/ERA5_pr_2021-01-01.nc'

In [ ]:
# Code example to read data from a netcdf file. 
# Some knowledge of the file structure and vaiables names in the file
# are needed. (They are ways to get this from within python)

# Set a filename: This is the file we just downloaded. 
filename = "ERA5_pr_2021-01-01.nc"

# Open the netcdf file
ds = nc.Dataset(filename,'r')

# Read the coodinate data
lons = ds.variables["longitude"][:]
lats = ds.variables["latitude"][:]
levs = ds.variables["level"][:]

nlon = len(lons)
nlat = len(lats)

# Read the temperature and wind data
k = 6         # choose which pressure level to read
print('Reading data at pressure:',levs[k])
temp = ds.variables["t"][0,k,:,:]
uwnd = ds.variables["u"][0,k,:,:]
vwnd = ds.variables["v"][0,k,:,:]

# It is good manners to close the file
ds.close()

print("Finished reading netcdf file.")


# Check the data

Convince yourself that the latitudes and longitudes were read in correctly. Print our their values to the screeed.
(You could do this with temperature as well, but it wil be many many values).


In [ ]:
# Print the dimensions to the screen
print("Number of longitudes: nlon=", nlon)
print("Number of latitudes : nlat=", nlat)

# Print the latitude and longitude values




##Construct a map of temperature

To convince yourself that the data is good, and get a view of the atmospheric conditions, construct a contour map of the data showing the temperature. We're using matplotlib to do the plotting. 

For geophysical data, we also need to set up a map projection. The easiest map projection is cylindrical equidistant: treating latitude and longitude as cartesian coordinates. This works find for many instances, but does not work well near the poles. 

As a bonus, you might try to overlay the wind data as vectors do visualize the airflow. Do do this you can use the quiver function.



In [ ]:
# Construct a map of the temperature data
#c = plt.contourf(lons,lats,temp)

# Try this to get vectors showing the streamlines
#c = plt.quiver(lons,lats,uwind,vwind)
c.show()



##Spherical coordinates
Notice the latitudes and longidtude spacing is constant. 
Anticipating what is coming below, we need to be mindful of the conversion between latitude and longitude to distances measured in meteres. 

*Calculate the distance between data points in metres.* 
Take care wth the geometry! In spheical coordinates while the distance between each latitude remains the same at each latitude, the distance between longitude points gets closer together toward the poles, and infact is exactly zero at the poles!

Some triganometry will be needed. Also, you will need to know te radius o fte earth $a = 6371000 m$.


In [ ]:
# Calulate the latitude and longitude grid spacing in units of metres

rearth = 6371000.                     # radius of the earth in metres
#coslat = np.cos(np.radians(lats))    # [HINT: cosine of latitude might be helpful]

dely = [Your code goes here]          # a constant value
delx = np.zeros(nlat)                 # depends on latitude

#Finite differences

The continuous derivative can be estimated as a finite approxmation using local "slopes". Consider the gridded data with values given by positions *i*: $ T_i = T(x_i)$.  The derivative at position *x_i* is then:


$$
  \frac{\partial T}{\partial x} \approx 
             \frac{ T_{i+1} - T_{i-1} }{2 \Delta x} 
$$

Notice that the gradient is "centered" on the interval between position $x_{i-1} and x_{i+1}$. This will be a problem at the ends of a domain, or in the case of time derivatives, where the "plus" values are not known. In that case a *one sided* estimate is needed. In the case of time with index $n$, we have 

$$
  \frac{\partial T}{\partial t} \approx
       \frac{ T_{n+1} - T_{n} }{\Delta t}  
$$



Construct the temperature tendency:
First, caluclate the gradient in T in each of the directions X and Y.
Then multiply by velocity to get the tendency. 

Note that it should have units of K/sec, which we might expect to be a small number. Expecting the temperature change to occure over 6 hours, multiple by $\Delta$t of 6 hours (in units of seconds). 

If we wanted to go further to make a 6 hour forecast, we could again use the finite difference method to make the prediction!

$$
 T(time+6 hours) = T(now) + \Delta T \frac{\partial T}{\partial t}
$$





In [ ]:
# Constrct the X and Y gradients

dTdx = [Your code goes here]

dTdy = [Your code goes here]

# Form the temperature time derivative ("tendency")

dTdt = [Your code goes here]

Final step - make a plot of the temperature change!

In [ ]:
#
# Make a contour map of forecast temperature CHANGE in units of K
#

dtime = 6*60*60       # 6 hours, in units of seconds


[Your code goes here]

#c.show()

##Outcome

Being able to obtain gridded datasets, visualize fields and manipulate the data is a fundemental activity in geophsyical fluids analysis. Having completed this notebook task, you've accomplished many essential building blocks on which more complex analysis can rely. 

All the of the modules used here have greater power than what we've looked at more details can be found at:

* [Numpy manual](https://numpy.org/doc/stable/user/)
* [NetCDF4 documentation](https://unidata.github.io/netcdf4-python/ )
* [Matplotlib](https://matplotlib.org/3.1.1/index.html)
* [Cartopy manual](https://scitools.org.uk/cartopy/docs/latest/)

There are many examples on these pages, which offers an excellent resources for more complicated coding.



In [79]:
# Clean up data file
!rm ERA5_pr_2021-01-01.nc